# Lab 1 - drzewa decyzyjne
*"pro" version*

Autorstwa:
- Wojciech Kot  151879
- Julia Samp    151775
z grupy L7, zajęcia piątkowe 11:45

Step - by - step - Wersja podstawowa (pro znajduje się poniżej)

In [2]:
import pandas

def prepare_pandas_df_for_basic_problem(path):
    data = pandas.read_csv(path)
    data = data.drop(['PassengerId', 'Name'], axis=1)
    data['Age'] = pandas.cut(data['Age'], bins=[0, 20, 40, 100], labels=['young', 'middle', 'old'], right=True)
    return data

# wariant bazowy, a więc wgrywamy do dataframe'u (bo już używamy pandas do czytania csv, żeby nie robić tego na piechotę)
# dane, z których wyrzucamy niepotrzebne kolumny PassengerId oraz Name, a następnie mapujemy wiek na 3 zmienne young, middle i old
#